In [ ]:
!pip install monai

: 

In [ ]:
import os
import nibabel as nib
import numpy as np
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Orientationd,
    ResizeWithPadOrCropd, NormalizeIntensityd, ToTensord
)
from monai.data import Dataset as MonaiDataset
from monai.networks.nets import UNet
from monai.losses import DiceCELoss
from monai.metrics import DiceMetric
from monai.inferers import sliding_window_inference
import matplotlib.pyplot as plt

In [ ]:
image_dir = "/kaggle/input/costa-adam/imagesTr"
label_dir = "/kaggle/input/costa-adam/labelsTr"
test_dir  = "/kaggle/input/costa-adam/imagesTs"
label_test_dir = "/kaggle/input/costa-adam/labelsTs"

image_paths = []
label_paths = []

for root, _, files in os.walk(image_dir):
    for file in files:
        if file.endswith(".nii") or file.endswith(".nii.gz"):
            image_paths.append(os.path.join(root, file))

for root, _, files in os.walk(label_dir):
    for file in files:
        if file.endswith(".nii") or file.endswith(".nii.gz"):
            label_paths.append(os.path.join(root, file))

image_paths.sort()
label_paths.sort()

print(f"Jumlah file image: {len(image_paths)}")
print(f"Jumlah file label: {len(label_paths)}")

In [ ]:
test_paths = sorted([
    os.path.join(test_dir, f) for f in os.listdir(test_dir)
    if f.endswith(".nii") or f.endswith(".nii.gz")
])
label_test_paths = sorted([
    os.path.join(label_test_dir, f) for f in os.listdir(label_test_dir)
    if f.endswith(".nii") or f.endswith(".nii.gz")
])

print(f"\nJumlah data test: {len(test_paths)}")
print(f"\nJumlah data test: {len(label_test_paths)}")

In [ ]:
input_size = (128, 128, 128)
train_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    ResizeWithPadOrCropd(keys=["image", "label"], spatial_size=input_size),
    NormalizeIntensityd(keys="image"),
    ToTensord(keys=["image", "label"])
])

In [ ]:
data_dicts = [{"image": img, "label": lbl} for img, lbl in zip(image_paths, label_paths)]
train_len = int(0.8 * len(data_dicts))
val_len = len(data_dicts) - train_len
train_files, val_files = random_split(data_dicts, [train_len, val_len])

train_ds = MonaiDataset(train_files, transform=train_transforms)
val_ds = MonaiDataset(val_files, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(
    spatial_dims=3, in_channels=1, out_channels=1,
    channels=(32, 64, 128, 256, 512),
    strides=(2, 2, 2, 2), num_res_units=2
).to(device)

loss_fn = DiceCELoss(sigmoid=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
best_loss, patience, trigger = float('inf'), 5, 0
train_losses = []
val_losses = []

In [ ]:
for epoch in range(1000):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        x, y = batch["image"].to(device), batch["label"].to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    train_losses.append(epoch_loss)

    val_loss = 0
    model.eval()
    with torch.no_grad():
        for val_batch in val_loader:
            val_x, val_y = val_batch["image"].to(device), val_batch["label"].to(device)
            val_pred = model(val_x)
            val_loss += loss_fn(val_pred, val_y).item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

    if val_loss < best_loss:
        best_loss = val_loss
        trigger = 0
        torch.save(model.state_dict(), "/kaggle/working/model.pt")
    else:
        trigger += 1
        if trigger >= patience:
            print("Early stopping.")
            break

In [ ]:
model.load_state_dict(torch.load("/kaggle/working/model.pt"))
model.eval()
metrics = {"Dice": [], "IoU": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []}

with torch.no_grad():
    for batch in val_loader:
        x = batch["image"].to(device)
        y_true = batch["label"].cpu().numpy().flatten()
        y_pred = torch.sigmoid(model(x)).cpu().numpy().flatten()
        y_bin = (y_pred > 0.5).astype(np.uint8)

        metrics["Dice"].append(f1_score(y_true, y_bin))
        metrics["IoU"].append(jaccard_score(y_true, y_bin))
        metrics["Accuracy"].append(accuracy_score(y_true, y_bin))
        metrics["Precision"].append(precision_score(y_true, y_bin))
        metrics["Recall"].append(recall_score(y_true, y_bin))
        metrics["F1"].append(f1_score(y_true, y_bin))

print("\nValidation Metrics:")
for k, v in metrics.items():
    print(f"{k}: {np.mean(v):.4f}")

In [ ]:
test_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    ResizeWithPadOrCropd(keys=["image", "label"], spatial_size=input_size),
    NormalizeIntensityd(keys="image"),
    ToTensord(keys=["image", "label"])
])

test_dicts = [{"image": img, "label": lbl} for img, lbl in zip(test_paths, label_test_paths)]
test_ds = MonaiDataset(test_dicts, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=1)

os.makedirs("/kaggle/working/predictions", exist_ok=True)
test_metrics = {"Dice": [], "IoU": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []}

for i, test_batch in enumerate(test_loader):
    x = test_batch["image"].to(device)
    y_true = test_batch["label"].cpu().numpy().flatten()

    with torch.no_grad():
        pred = sliding_window_inference(x, roi_size=input_size, sw_batch_size=1, predictor=model)
        pred_np = torch.sigmoid(pred).cpu().numpy()[0, 0]
        pred_bin = (pred_np > 0.5).astype(np.uint8)

    # Save prediction mask
    pred_nii = nib.Nifti1Image(pred_bin, affine=np.eye(4))
    nib.save(pred_nii, f"/kaggle/working/predictions/prediction_{i}.nii.gz")

    y_pred_flat = pred_bin.flatten()
    test_metrics["Dice"].append(f1_score(y_true, y_pred_flat))
    test_metrics["IoU"].append(jaccard_score(y_true, y_pred_flat))
    test_metrics["Accuracy"].append(accuracy_score(y_true, y_pred_flat))
    test_metrics["Precision"].append(precision_score(y_true, y_pred_flat))
    test_metrics["Recall"].append(recall_score(y_true, y_pred_flat))
    test_metrics["F1"].append(f1_score(y_true, y_pred_flat))

# Print test metrics
print("\nTest Metrics:")
for k, v in test_metrics.items():
    print(f"{k}: {np.mean(v):.4f}")

In [ ]:
plt.figure(figsize=(8, 5))
epochs = range(1, len(train_losses) + 1)
plt.plot(epochs, train_losses, label='Train Loss', marker='o')
plt.plot(epochs, val_losses, label='Validation Loss', marker='x')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Visualisasi beberapa sampel dari validation set
model.eval()
num_samples = 3  # jumlah volume untuk divisualisasi

with torch.no_grad():
    for i, batch in enumerate(val_loader):
        if i >= num_samples:
            break
        image, label = batch["image"].to(device), batch["label"].to(device)

        output = model(image)
        output = torch.sigmoid(output)
        pred = output > 0.5  # thresholding ke bentuk biner

        for j in range(image.shape[0]):
            img_np = image[j].cpu().numpy()[0]    # shape: (D, H, W)
            label_np = label[j].cpu().numpy()[0]
            pred_np = pred[j].cpu().numpy()[0]

            # Ambil irisan tengah volume (di sumbu depth/z)
            mid_slice = img_np.shape[0] // 2
            img_slice = img_np[mid_slice]
            label_slice = label_np[mid_slice]
            pred_slice = pred_np[mid_slice]

            # Visualisasi
            plt.figure(figsize=(12, 4))
            plt.subplot(1, 3, 1)
            plt.imshow(img_slice, cmap='gray')
            plt.title("Image (slice tengah)")
            plt.axis("off")

            plt.subplot(1, 3, 2)
            plt.imshow(label_slice, cmap='gray')
            plt.title("Ground Truth")
            plt.axis("off")

            plt.subplot(1, 3, 3)
            plt.imshow(pred_slice, cmap='gray')
            plt.title("Prediction")
            plt.axis("off")

            plt.tight_layout()
            plt.show()